In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

#### 데이터 명
- 지하철 데이터 : KR_TB_SBWY_STN_HR_CON_20190101_20190531_UTF8.csv
- 철도 데이터 : KR_TB_TR_STN_DAY_CON_20190101_20190531_UTF8.csv

In [2]:
train_df = pd.read_csv('KR_TB_TR_STN_DAY_CON_20200101_20200531_UTF8.csv')
train_df.head()

,RUN_DT,STN_CD,STN_CD_NM,ABRD_PRNB,GOFF_PRNB
0,20200101,924,송도교,2,-
1,20200101,3900023,서울,"38,306","50,440"
2,20200101,3900025,용산,"19,702","23,967"
3,20200101,3900030,영등포,"10,812","11,125"
4,20200101,3900039,안양,387,269


필요한 날짜 지역 하차 데이터 사용

In [3]:
df = train_df.loc[:, ['RUN_DT', 'STN_CD_NM', ' GOFF_PRNB ']]
df.head()

,RUN_DT,STN_CD_NM,GOFF_PRNB
0,20200101,송도교,-
1,20200101,서울,"50,440"
2,20200101,용산,"23,967"
3,20200101,영등포,"11,125"
4,20200101,안양,269


In [4]:
df.isnull().sum()

RUN_DT         0
STN_CD_NM      0
 GOFF_PRNB     0
dtype: int64

In [5]:
df['STN_CD_NM'].unique()

array(['송도교', '서울', '용산', '영등포', '안양', '수원', '오산', '서정리', '평택', '성환',
       '천안', '전의', '조치원', '부강', '신탄진', '대전', '옥천', '이원', '지탄', '심천', '영동',
       '황간', '추풍령', '김천', '구미', '사곡', '약목', '왜관', '신동', '대구', '동대구', '경산',
       '남성현', '청도', '상동', '밀양', '삼랑진', '원동', '물금', '화명', '구포', '사상', '부산',
       '덕소', '양평', '용문', '지평', '석불', '일신', '매곡', '양동', '삼산', '동화', '만종',
       '원주', '반곡', '신림', '제천', '단양', '희방사', '풍기', '영주', '안동', '의성', '탑리',
       '화본', '신녕', '북영천', '영천', '건천', '서경주', '서대전', '계룡', '연산', '논산',
       '강경', '함열', '익산', '김제', '신태인', '정읍', '백양사', '장성', '광주송정', '나주',
       '다시', '함평', '무안', '몽탄', '일로', '임성리', '목포', '삼례', '전주', '임실', '오수',
       '남원', '곡성', '구례구', '순천', '여천', '여수엑스포', '오송', '청주', '오근장', '청주공항',
       '증평', '음성', '주덕', '충주', '삼탄', '아산', '온양온천', '도고온천', '신례원', '예산',
       '삽교', '홍성', '광천', '청소', '대천', '웅천', '판교', '서천', '장항', '군산', '대야',
       '행신', '옥수', '왕십리', '청량리', '평내호평', '청평', '가평', '백양리', '강촌', '남춘천',
       '춘천', '옥산', '청리', '상주', '함창', '점촌', '용궁', '개

In [6]:
df[' GOFF_PRNB ']

0              - 
1         50,440 
2         23,967 
3         11,125 
4            269 
           ...   
35035          - 
35036         14 
35037          6 
35038          9 
35039        119 
Name:  GOFF_PRNB , Length: 35040, dtype: object

In [8]:
stations = ['송도교', '서울', '용산', '영등포', '안양', '수원', '오산', '서정리', '평택', '성환',
       '천안', '전의', '조치원', '부강', '신탄진', '대전', '옥천', '이원', '심천', '영동', '황간',
       '추풍령', '김천', '구미', '사곡', '약목', '왜관', '신동', '대구', '동대구', '경산',
       '남성현', '청도', '상동', '밀양', '삼랑진', '원동', '물금', '화명', '구포', '사상', '부산',
       '덕소', '양평', '용문', '지평', '석불', '일신', '매곡', '양동', '삼산', '동화', '만종',
       '원주', '반곡', '신림', '제천', '단양', '희방사', '풍기', '영주', '안동', '의성', '탑리',
       '화본', '신녕', '북영천', '영천', '건천', '서경주', '서대전', '계룡', '연산', '논산',
       '강경', '함열', '익산', '김제', '신태인', '정읍', '백양사', '장성', '광주송정', '나주',
       '다시', '함평', '무안', '몽탄', '일로', '임성리', '목포', '삼례', '전주', '임실', '오수',
       '남원', '곡성', '구례구', '순천', '여천', '여수엑스포', '오송', '청주', '오근장', '청주공항',
       '증평', '음성', '주덕', '충주', '삼탄', '아산', '온양온천', '신창', '도고온천', '신례원',
       '예산', '삽교', '홍성', '광천', '청소', '대천', '웅천', '판교', '서천', '장항', '군산',
       '대야', '행신', '옥수', '왕십리', '청량리', '동두천', '소요산', '전곡', '연천', '신망리',
       '대광리', '신탄리', '평내호평', '청평', '가평', '백양리', '강촌', '남춘천', '춘천', '옥산',
       '청리', '상주', '함창', '점촌', '용궁', '개포', '예천', '극락강', '광주', '봉화', '춘양',
       '현동', '분천', '양원', '승부', '석포', '철암', '동백산', '도계', '신기', '동해', '묵호',
       '망상해변', '정동진', '강릉', '쌍룡', '영월', '예미', '민둥산', '사북', '고한', '태백',
       '부전', '센텀', '신해운대', '기장', '좌천', '월내', '남창', '덕하', '태화강', '호계',
       '불국사', '경주', '안강', '하양', '한림정', '진영', '창원', '마산', '중리', '함안', '군북',
       '반성', '진주', '완사', '북천', '횡천', '하동', '진상', '광양', '벌교', '조성', '예당',
       '득량', '보성', '명봉', '이양', '능주', '화순', '효천', '서광주', '광명', '천안아산',
       '김천구미', '신경주', '울산', '상봉', '진례', '창원중앙', '인천공항T1', '백마고지', '공주',
       '포항', '횡성', '둔내', '평창', '진부', '인천공항T2', '월포', '장사', '강구', '영덕',
       '지탄', '각계', '문산', '임진강', '도라산', '초성리', '퇴계원', '사릉', '마석', '임기',
       '별어곡', '선평', '정선', '아우라지', '비동', '광운대', '의정부', '한탄강', '나전', '운천',
       '서빙고', '망우', '백운', '자미원', '의왕', '창동', '별내']

for i in range(0, 266):
    station = df[df['STN_CD_NM'] == stations[i]]
    station['GOFF_PRNB'] = station[' GOFF_PRNB '].str.strip()
    station.loc[station['GOFF_PRNB']=='-', 'GOFF_PRNB'] = 0
    station['GOFF_PRNB'].astype('float')
    
    print(stations[i], '\n', station['GOFF_PRNB'], '\n')

송도교 
 0        0
468      0
706      1
1871     2
2106     0
2342     0
2576     0
2812     0
3514     2
3751     1
3989     2
4224     2
4460     2
4929     2
5640     0
5873     0
6575     2
6812     0
8684     1
8916     0
9148     1
9615     1
10314    1
11475    2
12866    2
27711    0
Name: GOFF_PRNB, dtype: object 



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ValueError: could not convert string to float: '50,440'

In [ ]:
station.head()

In [ ]:
print(station.dtypes)